In [9]:
#=
Dr Keith Reid
Cailleach Computing Ltd
MIT license

Copyright © 2023 <copyright holders>

Permission is hereby granted, free of charge, to any person obtaining a copy of 
this software and associated documentation files (the “Software”), to deal in
the Software without restriction, including without limitation the rights to 
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of 
the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all 
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR 
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS 
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR 
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER 
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
=#

# s.0 libraries

using Test

# s.1 structs

mutable struct Element
    l_link::Char
    name::Char
    r_link::Char
end

# s.2 config

function get_cardinal()
    cardinal::Int64 = 10
    return cardinal
end

function get_universal_alphabet()
    universal_alphabet  = collect("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890")
    return universal_alphabet
end

# s.3 config

function amend(cardinal, place)
    while place < 1
        place = place + cardinal
    end
    while !(1 ≤ place ≤ cardinal)
        place = place - cardinal
    end
    return place
end

function ask_alphabet(place)
    cardinal            = get_cardinal()
    if cardinal ≤ 62
        universal_alphabet  = get_universal_alphabet()
        alphabet            = universal_alphabet[1:cardinal]
    else
        "ran out of letters"
    end
    char = alphabet[place]
end

function init_links(cardinal::Int64)
    links = Set()
    for i in 1:cardinal
        l_place         = amend(cardinal, i-1)
        name_place      = amend(cardinal, i)
        r_place         = amend(cardinal, i+1)        
        l_link          = ask_alphabet(l_place)        
        name            = ask_alphabet(name_place)
        r_link          = ask_alphabet(r_place)
        this_element    = Element(l_link, name, r_link)
        push!(links, this_element)
    end
    return links
end

# s.4 view

function scan(arbitrary, links)
    cardinal        = get_cardinal()
    this            = arbitrary
    is_next(x)      = x.name == this.r_link
    names           = Set()
    push!(names, this.name)
    for i in 1:cardinal-1
        next       = [x for x in collect(links) if is_next(x)][1]
        push!(names, next.name)
        this       = next
    end
    return names
end

scan (generic function with 1 method)

In [18]:
function wipe!(links, kill_name)
    is_left(x, kill_name)  = x.r_link  == kill_name
    is_kill(x, kill_name)  = x.name    == kill_name
    is_right(x, kill_name) = x.l_link  == kill_name
    left_element    = [x for x in collect(links) if is_left(x, kill_name)][1]
    kill_element    = [x for x in collect(links) if is_kill(x, kill_name)][1]
    right_element   = [x for x in collect(links) if is_right(x, kill_name)][1]
    post_wipe = Set()
    for i in collect(links)
        if i.name == left_element.name
            i.r_link = right_element.l_link
            push!(post_wipe, i)
        elseif i.name == right_element.name
            i.l_link = left_element.r_link
            push!(post_wipe, i)
        elseif i.name == kill_element.name
            nothing # push!(post_wipe, i)
        else
            push!(post_wipe, i)
        end
    end
    return post_wipe
end

function test_wipe!()
    links       = Set(Any[Element('h', 'i', 'j'), Element('e', 'f', 'g'), Element('a', 'b', 'c'), 
                          Element('c', 'd', 'e'), Element('j', 'a', 'b'), Element('f', 'g', 'h'), 
                          Element('b', 'c', 'd'), Element('g', 'h', 'i'), Element('d', 'e', 'f'), 
                          Element('i', 'j', 'a')])
    pre_names   = scan(Element('h', 'i', 'j'), links)
    println("pre_names:\t", pre_names)
    post_wipe   = wipe!(links, 'b')
    println("post_wipe:\t", post_wipe)
    post_names  = scan(Element('h', 'i', 'j'), post_wipe)
    @test post_names == (Any['f', 'j', 'a', 'i', 'c', 'd', 'e', 'g', 'h'])
    println("passed wipe")
end

test_wipe!()


pre_names:	Set(Any['f', 'j', 'a', 'i', 'c', 'd', 'e', 'g', 'h', 'b'])
post_wipe:	Set(Any[Element('d', 'e', 'f'), Element('e', 'f', 'g'), Element('i', 'j', 'a'), Element('c', 'd', 'e'), Element('f', 'g', 'h'), Element('b', 'c', 'd'), Element('j', 'a', 'b'), Element('h', 'i', 'j'), Element('g', 'h', 'i')])


LoadError: BoundsError: attempt to access 0-element Vector{Any} at index [1]

In [11]:
# s.5 control

function main_dancing_links()
    cardinal    = get_cardinal()
    links       = init_links(cardinal)
    rand_start  = Base.rand(links)
    names       = scan(rand_start, links)
    post_wipe   = wipe!(links, 'b')
end

main_dancing_links();

In [4]:
# s.6 tests

function test_ask_alphabet()
    place   = 3
    name    = ask_alphabet(place)
    @test name == 'c'
    @test typeof(name) == Char
    place   = 1
    name    = ask_alphabet(place)
    @test name == 'a'
    cardinal= 3
    place   = 0
    place   = amend(cardinal, place)
    name    = ask_alphabet(place)
    @test name == 'c'
    println("passed ask alphabet")
end

function test_amend()
    cardinal = 3
    place    = 1
    amended  = amend(cardinal, place)
    @test amended == 1
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 2
    amended  = amend(cardinal, place)
    @test amended == 2
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 3
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 4
    place    = 3
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 4
    amended  = amend(cardinal, place)
    @test amended == 1
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 5
    amended  = amend(cardinal, place)
    @test amended == 2
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 6
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 6
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 7
    amended  = amend(cardinal, place)
    @test amended == 1
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 0
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = -1
    amended  = amend(cardinal, place)
    @test amended == 2
    @test 1 ≤ amended ≤ cardinal
    cardinal = 7
    place    = -1
    amended  = amend(cardinal, place)
    @test amended == 6
    @test 1 ≤ amended ≤ cardinal
    cardinal = 7
    place    = 0
    amended  = amend(cardinal, place)
    @test amended == 7
    @test 1 ≤ amended ≤ cardinal
    println("passed test")
end

function test_init_links()
    cardinal = 3
    links    = init_links(cardinal)
    @test links == Set(Any[Element('a', 'b', 'c'), Element('b', 'c', 'a'), Element('c', 'a', 'b')])
    @test Element('a', 'b', 'c') ∈ links
    @test Element('b', 'c', 'a') ∈ links
    @test Element('c', 'a', 'b') ∈ links
    
    cardinal = 1
    links    = init_links(cardinal)
    @test links == Set(Any[Element('a', 'a', 'a')])
    
    cardinal = 2
    links    = init_links(cardinal)
    @test links == Set(Any[Element('a', 'b', 'a'), Element('b', 'a', 'b')])
    @test Element('a', 'b', 'a') ∈ links
    @test Element('b', 'a', 'b') ∈ links
end

function tests()
    println("testing")
    test_amend()
    test_init_links()
    test_ask_alphabet()
    println("passed all tests")
end

tests()

testing
passed test
Test Failed at In[4]:91
  Expression: links == Set(Any[Element('a', 'b', 'c'), Element('b', 'c', 'a'), Element('c', 'a', 'b')])
   Evaluated: Set(Any[Element('b', 'c', 'a'), Element('c', 'a', 'b'), Element('a', 'b', 'c')]) == Set(Any[Element('b', 'c', 'a'), Element('c', 'a', 'b'), Element('a', 'b', 'c')])


LoadError: [91mThere was an error during testing[39m